In [12]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
import numpy as np
import pandas as pd

chamada_api = yf.Ticker('PETR4.SA').history(period='5mo')


def plot_accumulation_distribution(df, ax):
    ad = (2*df.Close-df.High-df.Low) * df.Volume / (df.High - df.Low)
    ad.cumsum().ffill().plot(ax=ax, legend='Accum/Dist', color='#f00000')


def plot_bollinger_bands(df, ax):
    mean = df.Close.rolling(20).mean()
    stddev = df.Close.rolling(20).std()
    df['boll_hi'] = mean + 2.5*stddev
    df['boll_lo'] = mean - 2.5*stddev
    p0 = df.boll_hi.plot(ax=ax, color='#808080', legend='BB')
    p1 = df.boll_lo.plot(ax=ax, color='#808080')
    fplt.fill_between(p0, p1, color='#1A1C1D')


def plot_ema(df, ax):
    df.Close.ewm(span=9).mean().plot(ax=ax, legend='EMA',color = '#eef')


def plot_rsi(df, ax):
    diff = df.Close.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    n = 14
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    df['rsi'] = 100 - (100/(1+rs))
    df.rsi.plot(ax=ax, legend='RSI')
    fplt.set_y_range(-20, 120, ax=ax)
    fplt.add_band(3, 97, ax=ax, color='#1A1C1D')


def plot_moving_avg(df, ax):
    ma20 = df.Close.rolling(20).mean()
    ma50 = df.Close.rolling(50).mean()
    fplt.plot(ma20, legend = "MA-20", ax=ax)
    fplt.plot(ma50, legend = "MA-50", ax=ax)


def plot_candles(df, ax):
    candles = df[['Open', 'Close', 'High', 'Low']]
    daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
    daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))


def plot_volume(df, ax):
    df_renko = chamada_api.reset_index()
    fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=ax)
    fplt.plot(df_renko.Volume.ewm(span=24).mean(), ax=ax, color='#eef', legend='Volume')


ax,ax2,ax3,ax4 = fplt.create_plot(title='Gráfico de Velas', rows = 4, maximize=True)
b = fplt.background = fplt.odd_plot_background = '#010101'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'

plot_candles(chamada_api, ax)
plot_volume(chamada_api, ax2)
plot_bollinger_bands(chamada_api, ax)
plot_ema(chamada_api, ax)
plot_accumulation_distribution(chamada_api, ax3)
plot_rsi(chamada_api, ax4)
plot_moving_avg(chamada_api, ax)





# Plot the data on the graph


fplt.show()
